In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
url ="https://www.reddit.com/subreddits/"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [2]:
class listsubreddits:
    def __init__(self, url, numpag, head): #Inicia a classe para poder pegar a lista de subreddits com o numero de paginas a pegar
        self.initial_url = url
        self.url = url
        self.subrelist = []
        self.subs = 0
        self.numpag = numpag
        self.soup = None
        self.head = head
        
    def getreddit(self): #Pega o conteudo da pagina atual
        page = requests.get(self.url, self.head)
        self.soup = BeautifulSoup(page.content, 'html.parser')
    
    def findnexturl(self): #Encontra a url da proxima pagina
        self.url = self.soup.find("a", {"rel":"nofollow next"})["href"]
        
        
    def listsubreddit(self): #Separa o conteudo em uma lista de dicionarios para depois passar para um Dataframe
        for i in self.soup.find_all("div", {"data-type":"subreddit"}):
            self.subrelist.append({})
            self.subrelist[self.subs]["name"] = i.find("a", class_="title").get_text().split(":")[0][2:]
            self.subrelist[self.subs]["title"] = i.find("a", class_="title").get_text()
            try: 
                self.subrelist[self.subs]["description"] = i.find("p", class_=False).get_text()
            except:
                print(i.find("a", class_="title").get_text().split(":")[0][2:])
            assinantes = re.sub(",", "", i.find("span", class_="number").get_text())
            subage = re.search("\d+ year[s]?|\d+ month[s]?", i.find("p", class_="tagline").get_text()).group()
            self.subrelist[self.subs]["subscribed"] = int(assinantes)
            self.subrelist[self.subs]["age"] = subage
            self.subs += 1
            
    def kickstart(self): #Junta os outros metodos para ir adicionando os subreddits à lista e verificando se foi bloqueado pelo reddit
        for i in range(self.numpag):
            self.getreddit()
            print(" ")
            print(self.url)
            self.listsubreddit()
            try:
                self.findnexturl()
            except:
                print("reddit miseravel")
                break
            time.sleep(50)
        print("done")
            
    def getsubrelist(self): #Retorna a lista de subreddits
        return self.subrelist

In [3]:
data = listsubreddits(url, 5, headers)
data.kickstart()


 
https://www.reddit.com/subreddits/
 
https://www.reddit.com/subreddits/?User-Agent=Mozilla%2F5.0+%28Macintosh%3B+Intel+Mac+OS+X+10_10_1%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F39.0.2171.95+Safari%2F537.36&count=25&after=t5_2qh3s
reddit miseravel
done


In [8]:
df = pd.DataFrame(data.getsubrelist())
df.head(n=25)

,name,title,description,subscribed,age
0,news,"r/news: All news, US and international.","/r/news is: real news articles, primarily but ...",19080726,11 years
1,AskReddit,r/AskReddit: Ask Reddit...,r/AskReddit is the place to ask and answer tho...,24691079,11 years
2,worldnews,r/worldnews: World News,"A place for major news from around the world, ...",22246425,11 years
3,funny,r/funny: funny,Welcome to r/Funny: reddit's largest humour de...,26626622,11 years
4,relationship_advice,r/relationship_advice: Relationship Advice,Need help with your relationship? Whether it's...,2092428,10 years
5,politics,r/politics: Politics,/r/Politics is for news and discussion about U...,5458061,12 years
6,pics,r/pics: Reddit Pics,A place for pictures and photographs.,22923871,11 years
7,aww,r/aww: A subreddit for cute and cuddly pictures,"Things that make you go AWW! Like puppies, bun...",22094273,11 years
8,AmItheAsshole,r/AmItheAsshole: Am I the Asshole?,A catharsis for the frustrated moral philosoph...,1373362,6 years
9,gaming,r/gaming: r/gaming,A subreddit for (almost) anything related to g...,23779932,12 years


In [51]:
df.to_csv("subreddit_list.csv")

In [60]:
data.soup


<!DOCTYPE doctype html>

<html>
<head>
<title>Too Many Requests</title>
<style>
      body {
          font: small verdana, arial, helvetica, sans-serif;
          width: 600px;
          margin: 0 auto;
      }

      h1 {
          height: 40px;
          background: transparent url(//www.redditstatic.com/reddit.com.header.png) no-repeat scroll top right;
      }
    </style>
</head>
<body>
<h1>whoa there, pardner!</h1>
<p>we're sorry, but you appear to be a bot and we've seen too many requests
from you lately. we enforce a hard speed limit on requests that appear to come
from bots to prevent abuse.</p>
<p>if you are not a bot but are spoofing one via your browser's user agent
string: please change your user agent string to avoid seeing this message
again.</p>
<p>please wait 5 second(s) and try again.</p>
<p>as a reminder to developers, we recommend that clients make no
    more than <a href="http://github.com/reddit/reddit/wiki/API">one
    request every two seconds</a> to avoid se

In [45]:
df["name"].value_counts()

youseeingthisshit       1
whatisthisthing         1
PublicFreakout          1
insaneparents           1
worldnews               1
gaming                  1
AmItheAsshole           1
AskReddit               1
unpopularopinion        1
trashy                  1
pics                    1
relationship_advice     1
tifu                    1
Cringetopia             1
iamatotalpieceofshit    1
nba                     1
insanepeoplefacebook    1
aww                     1
interestingasfuck       1
facepalm                1
wholesomememes          1
dankmemes               1
memes                   1
BlackPeopleTwitter      1
leagueoflegends         1
Unexpected              1
Jokes                   1
WatchPeopleDieInside    1
explainlikeimfive       1
politics                1
mildlyinteresting       1
videos                  1
nottheonion             1
soccer                  1
NoStupidQuestions       1
therewasanattempt       1
Wellthatsucks           1
SquaredCircle           1
todayilearne

In [56]:
df.sort_values("subscribed", ascending = False).head(25)

,name,title,description,subscribed,age
3,funny,r/funny: funny,Welcome to r/Funny: reddit's largest humour de...,26624993,11 years
1,AskReddit,r/AskReddit: Ask Reddit...,r/AskReddit is the place to ask and answer tho...,24688784,11 years
9,gaming,r/gaming: r/gaming,A subreddit for (almost) anything related to g...,23778367,12 years
6,pics,r/pics: Reddit Pics,A place for pictures and photographs.,22922895,11 years
74,science,r/science: Science,This community is a place to share and discuss...,22476408,12 years
2,worldnews,r/worldnews: World News,"A place for major news from around the world, ...",22245291,11 years
7,aww,r/aww: A subreddit for cute and cuddly pictures,"Things that make you go AWW! Like puppies, bun...",22092891,11 years
20,todayilearned,r/todayilearned: Today I Learned (TIL),You learn something new every day; what did yo...,21591765,10 years
24,movies,r/movies: Movie News and Discussion,News & Discussion about Major Motion Pictures,21568377,11 years
17,videos,r/videos: /r/videos,The best place for video content of all kinds.,21356206,11 years


In [50]:
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
name           100 non-null object
title          100 non-null object
description    99 non-null object
subscribed     100 non-null int64
age            100 non-null object
dtypes: int64(1), object(4)
memory usage: 2.4+ KB


In [33]:
df["age"].nunique()

12